+--------------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+---+--------------+--------------------+--------------------+
|          cell|                 dob|               email|gender|                  id|            location|               login|                name|nat|         phone|             picture|          registered|
+--------------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+---+--------------+--------------------+--------------------+
|(839)-266-4634|{date -> 1952-12-...|michael.hughes@ex...|  male|{name -> , value ...|{country -> NULL,...|{sha1 -> 29eda632...|{last -> Hughes, ...| NZ|(039)-521-4310|{thumbnail -> htt...|{date -> 2012-03-...|
|06-44-56-84-52|{date -> 1980-04-...|maxence.vincent@e...|  male|{name -> INSEE, v...|{country -> NULL,...|{sha1 -> f3eaed22...|{last -> Vincent,...| FR|02-

In [5]:
from pyspark.sql import SparkSession

# Initialisez la session Spark
spark = SparkSession.builder.appName("KafkaConsumer").getOrCreate()

# Lisez les données en streaming à partir de Kafka
kafka_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "user_profiles") \
    .load()

# Affichez les données
query = kafka_stream \
    .selectExpr("CAST(value AS STRING)") \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()


ConnectionRefusedError: [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json
from pyspark.sql.types import StructType, StructField, StringType

# Créez une session Spark
spark = SparkSession.builder.appName("KafkaStreaming").getOrCreate()

# Définissez le schéma des données basé sur la structure de randomuser.me
schema = StructType([
    StructField("cell", StringType(), True),
    StructField("dob", StringType(), True),
    StructField("email", StringType(), True),
    # Ajoutez d'autres champs ici en fonction de la structure des données
])

# Lisez les données en streaming depuis Kafka
kafka_stream = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "user_profiles") \
    .load()

# Transformez les données JSON en utilisant le schéma défini
user_profiles = kafka_stream.selectExpr("CAST(value AS STRING) as value") \
    .select(from_json("value", schema).alias("data"))

# Affichez les données en streaming
query = user_profiles.writeStream.outputMode("append").format("console").start()
query.awaitTermination()


AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of Structured Streaming + Kafka Integration Guide.